In [1]:
import os
import torch
from models import SpecialFuseNet, SpecialFuseNetModel
from data_manager import rgbd_gradients_dataset, rgbd_gradients_dataloader
from torchvision import transforms as T


In [2]:
fusenet = SpecialFuseNet()

In [3]:
print(fusenet)

SpecialFuseNet(
  (CBR1_RGB_ENC): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
  )
  (RGB_POOL1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (CBR2_RGB_ENC): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
  )
  (RGB_POOL2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, c

In [4]:
CWD             = os.getcwd()
DATASET_DIR     = os.path.join(CWD,'data/nyuv2')
print(DATASET_DIR)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

/home/manor/cs236781-DeepLearning/project/data/nyuv2
Using device: cpu


In [5]:
IMAGE_SIZE = (64,64)
TRAIN_TEST_RATIO = 0.9
BATCH_SIZE=4
NUM_WORKERS=4

In [6]:
tf = T.Compose([
    # Resize to constant spatial dimensions
    T.Resize(IMAGE_SIZE),
    # PIL.Image -> torch.Tensor
    T.ToTensor(),
    # Dynamic range [0,1] -> [-1, 1]
    T.Normalize(mean=(.5,.5,.5), std=(.5,.5,.5)),
])

In [7]:
rgbd_grads_ds = rgbd_gradients_dataset(root=DATASET_DIR,transforms=tf)

In [8]:
dl_train,dl_test = rgbd_gradients_dataloader(root=DATASET_DIR,
                                             batch_size=BATCH_SIZE,
                                             num_workers=NUM_WORKERS,
                                             train_test_ration=TRAIN_TEST_RATIO, transforms=tf)

In [9]:
sample_batch = next(iter(dl_test))
print(sample_batch.keys())
print(sample_batch['rgb'].shape, sample_batch['rgb'].min(),sample_batch['rgb'].max(),
      sample_batch['rgb'].mean(), sample_batch['rgb'].std())
print(sample_batch['depth'].shape, sample_batch['depth'].min(),sample_batch['depth'].max(),
      sample_batch['depth'].mean(), sample_batch['depth'].std())
print(sample_batch['x'].shape, sample_batch['x'].min(),sample_batch['x'].max(),
      sample_batch['x'].mean(), sample_batch['x'].std())
print(sample_batch['y'].shape, sample_batch['y'].min(),sample_batch['y'].max(),
      sample_batch['y'].mean(), sample_batch['y'].std())
rgb_size = tuple(sample_batch['rgb'].shape[1:])
depth_size = tuple(sample_batch['depth'].shape[1:])
grads_size = tuple(sample_batch['x'].shape[1:])
print(rgb_size)
print(depth_size)
print(grads_size)

dict_keys(['rgb', 'depth', 'x', 'y'])
torch.Size([4, 3, 64, 64]) tensor(-0.9922) tensor(1.) tensor(-0.2194) tensor(0.5509)
torch.Size([4, 1, 64, 64]) tensor(-1.) tensor(0.9686) tensor(0.0178) tensor(0.5348)
torch.Size([4, 1, 64, 64]) tensor(-1., dtype=torch.float64) tensor(1., dtype=torch.float64) tensor(-0.0601, dtype=torch.float64) tensor(0.1949, dtype=torch.float64)
torch.Size([4, 1, 64, 64]) tensor(-1., dtype=torch.float64) tensor(1., dtype=torch.float64) tensor(0.0307, dtype=torch.float64) tensor(0.2334, dtype=torch.float64)
(3, 64, 64)
(1, 64, 64)
(1, 64, 64)


In [10]:
fusenetmodel = SpecialFuseNetModel(device=device,rgb_size=rgb_size,depth_size=depth_size,grads_size=grads_size)

[debug] - in initialize
[debug] - child=CBR1_RGB_ENC
[debug] - child=RGB_POOL1
[debug] - child=CBR2_RGB_ENC
[debug] - child=RGB_POOL2
[debug] - child=CBR3_RGB_ENC
[debug] - child=RGB_POOL3
[debug] - child=RGB_DROPOUT3
[debug] - child=CBR4_RGB_ENC
[debug] - child=RGB_POOL4
[debug] - child=RGB_DROPOUT4
[debug] - child=CBR5_RGB_ENC
[debug] - child=RGB_POOL5
[debug] - child=RGB_DROPOUT5
[debug] - child=CBR1_DEPTH_ENC
[debug] - child=DEPTH_POOL1
[debug] - child=CBR2_DEPTH_ENC
[debug] - child=DEPTH_POOL2
[debug] - child=CBR3_DEPTH_ENC
[debug] - child=DEPTH_POOL3
[debug] - child=DEPTH_DROPOUT3
[debug] - child=CBR4_DEPTH_ENC
[debug] - child=DEPTH_POOL4
[debug] - child=DEPTH_DROPOUT4
[debug] - child=CBR5_DEPTH_ENC
[debug] - child=UNPOOL5
[debug] - child=CBR5_RGB_DEC
Network initialize with xavier ... 
		[debug] - classname=Conv2d
		[debug] - initialize weight
		[debug] - initialize bias
Done.
Network initialize with xavier ... 
		[debug] - classname=BatchNorm2d
		[debug] - initialize weight&bia